In [8]:
import numpy as np
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize,RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
from sklearn.naive_bayes import MultinomialNB
from collections import Counter
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from nltk.sentiment.util import mark_negation
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression


In [9]:

def load_pickle(filename):
    with open(filename,"rb") as f:
        dump=pickle.load(f)
    return dump

def dump_pickle(dump,filename):
    with open(filename,"wb") as f:
        pickle.dump(dump,f)


In [10]:
train_file='data/A1_Data/toy.json'
dev_file='data/A1_Data/toy.json'
tokenized_train="tokenized_train.pkl"
tokenized_dev="tokenized_dev.pkl"
log=10**5

In [11]:
train_file='data/A1_Data/train.json'
dev_file='data/A1_Data/dev.json'

In [12]:
def read_data(filename):
    x=[]
    y=[]
    
    file_reader=open(filename,"r")
    for line in file_reader:
        mapping=json.loads(line)
        x.append(mapping['review'])
        y.append(mapping['ratings'])
    
    return x,y

In [13]:
train_sent,train_y=read_data(train_file)
dev_sent,dev_y=read_data(dev_file)
print(len(train_sent),len(dev_sent))

1000000 200000


## Tokenizer

In [92]:
def tokenize_data(data):
    tokenized=[]
    for i in range(len(data)):
        if(i%100000==0):
            print(i)
        tokenized.append(word_tokenize(data[i]))
    return tokenized

In [15]:
# train_x=tokenize_data(train_sent)
# dev_x=tokenize_data(dev_sent)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
0
100000


In [8]:
# print(len(train_x),train_x[0],type(train_x),tokenized_train)

In [9]:
# dump_pickle(train_x,tokenized_train)
# dump_pickle(dev_x,tokenized_dev)

In [152]:
word_tokenize("I am ANkesh. I am 21 :)")

['I', 'am', 'ANkesh', '.', 'I', 'am', '21', ':', ')']

## Loader

In [153]:
train_x=load_pickle(tokenized_train)
dev_x=load_pickle(tokenized_dev)

In [154]:
print(len(train_x),len(dev_x))

1000000 200000


In [49]:
stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer() 
#Consider Stemming

In [50]:
def rem_stopwords_lemmatize(data):
    new_data=[]
    count=0
    for line in data:
        count+=1
        if(count%log==0):
            print(count)
        line=mark_negation(line,shallow=True,double_neg_flip=True)
        new_line=[]
        for word in line:
            lemmatized_word=lemmatizer.lemmatize(word)
            if lemmatized_word not in stop_words:
                new_line.append(lemmatized_word)
        new_data.append(new_line)
    return new_data

In [51]:
# train_x=rem_stopwords_lemmatize(train_x)
# dev_x=rem_stopwords_lemmatize(dev_x)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
100000
200000


## Save Cleaned

In [52]:
# dump_pickle(train_x,'cleaned_train.pkl')
# dump_pickle(dev_x,'cleaned_dev.pkl')mean_squared_error(y_true, y_pred)

## Load Cleaned

In [14]:
train_x_comp=load_pickle('cleaned_train.pkl')
dev_x_comp=load_pickle('cleaned_dev.pkl')

In [161]:
# cut=50000
# train_x=train_x_comp[0:cut]
# train_y=train_y[0:cut]
# dev_x=dev_x_comp[0:cut]
# dev_y=dev_y[0:cut]

In [19]:
print(len(train_x_comp),len(dev_x_comp))
print(isinstance(train_x_comp[0][0],str))
for i in train_x_comp:
    for j in i:
        if(isinstance(j,str)==False):
            print(j)

1000000 200000
True


In [20]:
# Helper:https://www.ritchieng.com/machine-learning-multinomial-naive-bayes-vectorization/

In [22]:
def temp(x):
    return x

In [24]:
vect = CountVectorizer(analyzer=temp)
print(vect)
train_x_features = vect.fit_transform(train_x_comp)
dev_x_features = vect.transform(dev_x_comp)
# print(train_x_features,dev_x_features)0.5605887140074481

CountVectorizer(analyzer=<function temp at 0x7f071b8ba0d0>, binary=False,
        decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocessor=None,
        stop_words=None, strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None)


In [25]:
train_x_features,dev_x_features

(<1000000x663426 sparse matrix of type '<class 'numpy.int64'>'
 	with 64420201 stored elements in Compressed Sparse Row format>,
 <200000x663426 sparse matrix of type '<class 'numpy.int64'>'
 	with 12809187 stored elements in Compressed Sparse Row format>)

In [26]:
def get_accuracy(gold,pred):
    length=len(gold)
    return np.sum(gold==pred)*1.0/length
def get_Fscore(gold,pred):
    return (f1_score(gold, pred, average='macro'),f1_score(gold, pred, average='micro'))      

## Multinomial Naive Bayes

In [27]:
def MultinomialNaiveBayes(X,Y,dev_X,dev_Y):
    nb = MultinomialNB()
    %time nb.fit(X,Y)
    pred_X=nb.predict(X)
    pred_dev=nb.predict(dev_X)
    print(get_accuracy(Y,pred_X))
    print(get_accuracy(dev_y,pred_dev))
    print(get_Fscore(Y,pred_X))
    print(get_Fscore(dev_y,pred_dev))
    print(mean_squared_error(dev_y,pred_dev))
    print(mean_squared_error(dev_y,pred_dev))
    print(confusion_matrix(Y, pred_X))
    print(confusion_matrix(dev_y, pred_dev))
    return nb

## SVM

In [28]:
def SVM(X,Y,dev_X,dev_Y):
    clf = LinearSVC(random_state=0, tol=1e-5,max_iter=1000)
    %time clf.fit(X,Y)
    pred_X=clf.predict(X)
    pred_dev=clf.predict(dev_X)
    print(get_accuracy(Y,pred_X))
    print(get_accuracy(dev_y,pred_dev))
    print(get_Fscore(Y,pred_X))
    print(get_Fscore(dev_y,pred_dev))
    print(mean_squared_error(dev_y,pred_dev))
    print(confusion_matrix(Y, pred_X))
    print(confusion_matrix(dev_y, pred_dev))
    return clf

In [133]:
selection_obj=SelectKBest(chi2, k=1000)
X_new = selection_obj.fit_transform(train_x_features, train_y)
dev_new=selection_obj.transform(dev_x_features)
X_new,dev_new

## Logistic Regression

In [29]:
def logsiticRegression(X,Y,dev_X,dev_Y):
    log_reg = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial', max_iter=100).fit(X, Y)
    pred_X=log_reg.predict(X)
    pred_dev=log_reg.predict(dev_X)
    print(get_accuracy(Y,pred_X))
    print(get_accuracy(dev_y,pred_dev))
    print(get_Fscore(Y,pred_X))
    print(get_Fscore(dev_y,pred_dev))
    print(mean_squared_error(dev_y,pred_dev))
    print(confusion_matrix(Y, pred_X))
    print(confusion_matrix(dev_y, pred_dev))
    return log_reg 

## Linear Regression

In [35]:
def linearRegression(X,Y,dev_X,dev_Y):
    reg = LinearRegression().fit(X, Y)
    pred_X=reg.predict(X)
    pred_dev=reg.predict(dev_X)
    print(mean_squared_error(dev_y,pred_dev))
    return reg

## RUN

In [37]:
cut=20000
linearRegression(train_x_features[0:cut],train_y[0:cut],dev_x_features[0:cut],dev_y[0:cut])

(20000, 663426)
20000
(20000, 663426)
20000


ValueError: Found input variables with inconsistent numbers of samples: [200000, 20000]